In [62]:
from openai import OpenAI
#load csv
import pandas as pd
df_master = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\merged_leads.csv')
df = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\cleaned_leads.csv')

client = OpenAI()


filter_columns = [
    'Lead Number','Lead Source', 'Lead Origin', 'Do Not Email', 'Do Not Call',
    'Converted', 'Country', 'Lead Stage', 'City'
]

def generate_schema(df, columns):
    schema = []
    for col in columns:
        if col not in df.columns:
            continue
        dtype = str(df[col].dtype)
        examples = df[col].dropna().unique()[:5]  # Limit to 5 unique examples
        example_values = ", ".join([str(e) for e in examples])
        schema.append(f"- {col} ({dtype}): e.g., {example_values}")
    return "\n".join(schema)

schema_text = generate_schema(df, filter_columns)

print("Schema for LLM:\n")
print(schema_text)


Schema for LLM:

- Lead Number (int64): e.g., 660030, 659357, 655510, 655287, 654061
- Lead Source (object): e.g., Organic Search, Google, Direct Traffic, Olark Chat, Reference
- Lead Origin (object): e.g., API, Landing Page Submission, Lead Add Form
- Do Not Email (object): e.g., No, Yes
- Do Not Call (object): e.g., No
- Converted (int64): e.g., 0, 1
- Country (object): e.g., India, United States
- Lead Stage (object): e.g., Unreachable, Not Interested, Qualified, Closed, Interested
- City (object): e.g., Select, Mumbai, Thane & Outskirts, Other Metro Cities, Other Cities of Maharashtra


In [63]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_pandas_query",
            "description": "Run a pandas query string on a DataFrame. Use 'df' as the variable name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_str": {
                        "type": "string",
                        "description": "The Python code string to evaluate on the DataFrame. Use 'df' as the DataFrame variable."
                    }
                },
                "required": ["query_str"]
            }
        }
    }
]

In [ ]:
from openai import OpenAI
#load csv
import pandas as pd
df_master = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\merged_leads.csv')
df = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\cleaned_leads.csv')

client = OpenAI()


filter_columns = [
    'Lead Number','Lead Source', 'Lead Origin', 'Do Not Email', 'Do Not Call',
    'Converted', 'Country', 'Lead Stage', 'City'
]

def generate_schema(df, columns):
    schema = []
    for col in columns:
        if col not in df.columns:
            continue
        dtype = str(df[col].dtype)
        examples = df[col].dropna().unique()[:5]  # Limit to 5 unique examples
        example_values = ", ".join([str(e) for e in examples])
        schema.append(f"- {col} ({dtype}): e.g., {example_values}")
    return "\n".join(schema)

schema_text = generate_schema(df, filter_columns)

print("Schema for LLM:\n")
print(schema_text)

tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_pandas_query",
            "description": "Run a pandas query string on a DataFrame. Use 'df' as the variable name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_str": {
                        "type": "string",
                        "description": "The Python code string to evaluate on the DataFrame. Use 'df' as the DataFrame variable."
                    }
                },
                "required": ["query_str"]
            }
        }
    }
]
import json
def execute_pandas_query(df, query_str):
    local_vars = {"df": df.copy()}
    try:
        return eval(query_str, {"__builtins__": {}}, local_vars)
    except Exception as e:
        print(f"Error: {e}")
        return None
def handle_tool_call(tool_call, df):
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if function_name == "execute_pandas_query":
        query_str = arguments["query_str"]
        result = execute_pandas_query(df, query_str)

        if isinstance(result, pd.DataFrame):
            return result.to_dict(orient="records")
        elif isinstance(result, pd.Series):
            return result.tolist()
        else:
            return result
def nl_to_query(nl_query):
    print("Entered NL query:", nl_query)
    schema_description = generate_schema(df, filter_columns)
    prompt= f"""
    You are an expert data analyst.

    You have access to a tool called `execute_pandas_query` that accepts a Python expression as input and runs it on a DataFrame called `df`.

    Here is the DataFrame schema:
    {schema_description}

    Now, a user has asked to filter the DataFrame with this request:
    '{nl_query}'

    You should call the tool with the appropriate pandas filtering expression using `df`.

    Only call the tool. Do not answer the query yourself.


    """

    messages=[
        {"role": "system", "content": "You are a pandas expert."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
            messages=messages,
            model="gpt-4o",
            stream=False,
            tool_choice="auto",
            tools=tools,
            temperature=0, # Keep temperature low for deterministic SQL generation
        )
    try:
        print(response.choices[0].message.content.strip())
    except Exception as e:
        print(f"Error: {e}")
       
    tool_call = response.choices[0].message.tool_calls[0]
    print(f"Tool call: {tool_call}")
    tool_result = handle_tool_call(tool_call, df)
    print(tool_result)
    return tool_result
    # df_query = response.choices[0].message.content.strip().replace("\n", " ")
    followup = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                *messages,
                {
                    "role": "assistant",
                    "tool_calls": [tool_call]
                },
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": "execute_pandas_query",
                    "content": json.dumps(tool_result)
                }
            ]
        )
    return (followup.choices[0].message.content.strip())
    # return df_query
nl_query = "Can you get me leads from Bombay who haven’t converted?"
code = nl_to_query(nl_query)
# print(code)


# execute_pandas_query(df, code)

Entered NL query: Can you get me leads from Bombay who haven’t converted?
Error: 'NoneType' object has no attribute 'strip'
Tool call: ChatCompletionMessageToolCall(id='call_bscs5fGcoVXomzPsrvO0afXK', function=Function(arguments='{"query_str":"df[(df[\'City\'] == \'Mumbai\') & (df[\'Converted\'] == 0)]"}', name='execute_pandas_query'), type='function')
[{'Lead Number': 659357, 'Lead Source': 'Google', 'Lead Origin': 'Landing Page Submission', 'Do Not Email': 'No', 'Do Not Call': 'No', 'Converted': 0, 'Country': 'India', 'Mobile Number': nan, 'Lead Stage': 'Unreachable', 'Lead Grade': nan, 'Lead Score': 120, 'Age': 'Select', 'Email': 'Yogeshsadarang@yahoo.in', 'City': 'Mumbai'}, {'Lead Number': 655287, 'Lead Source': 'Direct Traffic', 'Lead Origin': 'Landing Page Submission', 'Do Not Email': 'No', 'Do Not Call': 'No', 'Converted': 0, 'Country': 'India', 'Mobile Number': nan, 'Lead Stage': 'Not Interested', 'Lead Grade': nan, 'Lead Score': 90, 'Age': 'Select', 'Email': 'vkaradkar@in.imsh

In [102]:
code

[{'Lead Number': 659357,
  'Lead Source': 'Google',
  'Lead Origin': 'Landing Page Submission',
  'Do Not Email': 'No',
  'Do Not Call': 'No',
  'Converted': 0,
  'Country': 'India',
  'Mobile Number': nan,
  'Lead Stage': 'Unreachable',
  'Lead Grade': nan,
  'Lead Score': 120,
  'Age': 'Select',
  'Email': 'Yogeshsadarang@yahoo.in',
  'City': 'Mumbai'},
 {'Lead Number': 655287,
  'Lead Source': 'Direct Traffic',
  'Lead Origin': 'Landing Page Submission',
  'Do Not Email': 'No',
  'Do Not Call': 'No',
  'Converted': 0,
  'Country': 'India',
  'Mobile Number': nan,
  'Lead Stage': 'Not Interested',
  'Lead Grade': nan,
  'Lead Score': 90,
  'Age': 'Select',
  'Email': 'vkaradkar@in.imshealth.com',
  'City': 'Mumbai'},
 {'Lead Number': 615883,
  'Lead Source': 'Google',
  'Lead Origin': 'Landing Page Submission',
  'Do Not Email': 'No',
  'Do Not Call': 'No',
  'Converted': 0,
  'Country': 'India',
  'Mobile Number': nan,
  'Lead Stage': 'Not Interested',
  'Lead Grade': nan,
  'Lead S

In [103]:
tools_1 = [

    {
  "type": "function",
  "function": {
    "name": "nl_to_query",
    "description": "Convert a natural language query into a valid pandas filtering expression.",
    "parameters": {
      "type": "object",
      "properties": {
        "nl_query": {
          "type": "string",
          "description": "The user's natural language filter request (e.g., 'show leads from Delhi with conversion 0')."
        }
      },
      "required": ["nl_query"]
    }
  }
}

]

In [104]:
system_instructions = """
You are a friendly and helpful assistant for a Sales Development Representative (SDR).

Your primary responsibility is to help the SDR filter and identify relevant leads and write personalized sales emails.

You have access to one tool:
1. `nl_to_query` – USe this to access the leads database. You need to input the SDR's request in natural language, and it will convert it into a pandas query string.


Do not try to write the pandas query yourself.
Always use the `nl_to_query` tool to translate the manager’s request to retrieve the relevant data.
"""
assistant = client.beta.assistants.create(
  name="Sales development representative Assistant",
  instructions=system_instructions,
  model="gpt-4o",
  tools=tools_1,
  temperature=0,
)

In [116]:
assistant.id

'asst_AkGISgAeCdyXen8cGLkpP8lO'

In [115]:
def chat_once(assistant_id, user_input):
    # 1. Create a thread
    thread = client.beta.threads.create()

    # 2. Add user message
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )

    # 3. Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

    # 4. Poll for status until tool is called or run completes
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status == "requires_action":
            break
        elif run_status.status == "completed":
            # No tool call, assistant finished directly
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            for msg in reversed(messages.data):
                if msg.role == "assistant":
                    print("Assistant:", msg.content[0].text.value.strip())
                    return


    # 5. Handle tool call(s)
    tool_outputs = []
    for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)

        if name == "nl_to_query":
            output = nl_to_query(args["nl_query"])
        else:
            output = f"Unsupported tool: {name}"

        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": json.dumps(output)
        })

    # 6. Submit tool outputs
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

    # 7. Wait for final assistant response
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status == "completed":
            break
        elif run_status.status in ["failed", "cancelled"]:
            raise RuntimeError("Run failed after tool call")


    # 8. Print final assistant message
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for msg in reversed(messages.data):
        if msg.role == "assistant":
            print("Assistant:", msg.content[0].text.value.strip())
            break

In [117]:
chat_once(assistant_id=assistant.id, user_input="Get me leads from Bombay who haven’t converted.")


Entered NL query: leads from Bombay who haven’t converted
Error: 'NoneType' object has no attribute 'strip'
Tool call: ChatCompletionMessageToolCall(id='call_fDNiNkaTXbkxT8zyvk5qjrR7', function=Function(arguments='{"query_str":"df[(df[\'City\'] == \'Mumbai\') & (df[\'Converted\'] == 0)]"}', name='execute_pandas_query'), type='function')
[{'Lead Number': 659357, 'Lead Source': 'Google', 'Lead Origin': 'Landing Page Submission', 'Do Not Email': 'No', 'Do Not Call': 'No', 'Converted': 0, 'Country': 'India', 'Mobile Number': nan, 'Lead Stage': 'Unreachable', 'Lead Grade': nan, 'Lead Score': 120, 'Age': 'Select', 'Email': 'Yogeshsadarang@yahoo.in', 'City': 'Mumbai'}, {'Lead Number': 655287, 'Lead Source': 'Direct Traffic', 'Lead Origin': 'Landing Page Submission', 'Do Not Email': 'No', 'Do Not Call': 'No', 'Converted': 0, 'Country': 'India', 'Mobile Number': nan, 'Lead Stage': 'Not Interested', 'Lead Grade': nan, 'Lead Score': 90, 'Age': 'Select', 'Email': 'vkaradkar@in.imshealth.com', 'Cit

In [108]:
run_status.required_action.submit_tool_outputs.tool_calls

[RequiredActionFunctionToolCall(id='call_Axuyxb4sg4paKYS0b7Cvp8Lk', function=Function(arguments='{"nl_query":"leads from Delhi who haven’t converted"}', name='nl_to_query'), type='function')]

In [107]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

print(run_steps)

SyncCursorPage[RunStep](data=[RunStep(id='step_50xsd8b8nH6EHzUIWPOAf8Bv', assistant_id='asst_AkGISgAeCdyXen8cGLkpP8lO', cancelled_at=None, completed_at=None, created_at=1744451264, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_kCK6ciej3sl3RgEhmzlY0IX6', status='in_progress', step_details=ToolCallsStepDetails(tool_calls=[FunctionToolCall(id='call_YUul3fOXil784Q84nTZDfOua', function=Function(arguments='{"nl_query":"leads from Bombay who haven\'t converted"}', name='nl_to_query', output=None), type='function')], type='tool_calls'), thread_id='thread_WSZcdq37VBnMFb8yCLNhPfEB', type='tool_calls', usage=None, expires_at=1744451863)], has_more=False, object='list', first_id='step_50xsd8b8nH6EHzUIWPOAf8Bv', last_id='step_50xsd8b8nH6EHzUIWPOAf8Bv')


In [74]:
import time
def chat_with_assistant(assistant_id, thread_id=None):
    if thread_id is None:
        thread = client.beta.threads.create()
        thread_id = thread.id
        print(f"Started new thread: {thread_id}")
    else:
        print(f"Resuming thread: {thread_id}")
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in {"exit", "quit"}:
            print("Goodbye!")
            break

        # Add user message to thread
        client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_input
        )

        # Run assistant
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id
        )

        # Wait for assistant to respond
        while True:
            run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
            if run_status.status == "completed":
                break
            elif run_status.status in ["failed", "cancelled"]:
                print(f"Run {run_status.status}")
                return
            time.sleep(1)

        # Print assistant reply
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        for msg in reversed(messages.data):
            if msg.role == "assistant":
                print("\nAssistant:", msg.content[0].text.value.strip())
                break

In [75]:
chat_with_assistant(assistant_id="asst_eXrmKqet29Vk6hRpX5xUOf4N")  # replace with your assistant ID


Started new thread: thread_5emVJDzlRNAZpxNB4hzVimTg

Assistant: Could you please provide me with the schema or description of the DataFrame containing the leads information? This should include the column names and example values, so I can filter the leads from Mumbai.


BadRequestError: Error code: 400 - {'error': {'message': 'Message content must be non-empty.', 'type': 'invalid_request_error', 'param': 'content', 'code': None}}